In [6]:
#!pip install pymysql
#!pip install pymssql

import pandas as pd
import LibData
import LibUtils

#jobName = '201216_Train_pilot'
jobName = '201216_Train_split1'
x, y = LibData.GetXY3(jobName)

# 스케일링
cols_toScaling = ['ElapsedMth', 'ContAge', 'Sex', 'ProdCode', 'Prem', 'PremYr', 'AccumPrem', 'ContAmt', 'StartAge', 'SumFund', 'FundVal01', 'FundVal02', 'FundVal03', 'FundAllo01', 'FundAllo02', 'FundAllo03', 'IR01', 'IR02', 'IR03', 'IR04', 'IR05', 'IR06', 'IR07', 'IR08', 'IR09', 'IR10', 'IR11', 'IR12', 'IR13', 'IR14']
model, x_s = LibUtils.ScaleStandard(x[cols_toScaling])

# 원핫인코딩은 스케일링 하면 안됨
cols_toOneHot = ['ProdCode']
x_o = pd.get_dummies(x[cols_toOneHot], columns=cols_toOneHot, prefix='prd')

x = pd.concat([x_s, x_o], axis=1)

from sklearn.preprocessing import PolynomialFeatures
# 3차원 하면 램이 뻗음, 2차원까지만 할 수 있음
x_poly = PolynomialFeatures(2).fit_transform(x)

In [7]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import numpy as np

tuned_parameters = {
    'alpha' : [1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]
    #rg = [np.float_power(10, i) for i in np.array([range(-2, 6)])]
}


gridSearcher = GridSearchCV(Ridge(),
                            param_grid = tuned_parameters, 
                            cv=5, 
                            scoring='neg_mean_absolute_error',
                            verbose=2)

gridSearcher.fit(x_poly, y['pv_claim'])

#'Temp','Temp',
LibUtils.WriteNote('T_ParamSearch_Ridge','grid_cv5_data10_negmae_poly2dim',
                   [gridSearcher.param_grid, gridSearcher.best_params_, gridSearcher.best_score_])

LibUtils.ELoop()

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] alpha=0.01 ......................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....................................... alpha=0.01, total=   4.3s
[CV] alpha=0.01 ......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s


[CV] ....................................... alpha=0.01, total=   4.0s
[CV] alpha=0.01 ......................................................
[CV] ....................................... alpha=0.01, total=   4.0s
[CV] alpha=0.01 ......................................................
[CV] ....................................... alpha=0.01, total=   4.0s
[CV] alpha=0.01 ......................................................


KeyboardInterrupt: ignored